In [ ]:
import networkx as nx
import operator
import pylab
import random
import sys

import time
import numpy as np
import osmnx as ox
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import clear_output
import matplotlib.cm as cm
import matplotlib.colors as colors
from IPython.display import Image
from pprint import pprint
import folium

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def robustness_analysis(g, node_classifier, recalculate=False):
    m = node_classifier(g)
    l = sorted(m.items(), key=operator.itemgetter(1), reverse=True)
    x = []
    y = []
    
    
    sbg = [g.subgraph(c).copy() for c in nx.connected_components(g)]
    largest_component = max(sbg, key=lambda c: sum(d["length"] for u, v, d in c.edges(data=True)))
    
    
    n = len(g.nodes())
    tl=sum(d["length"] for u, v, d in g.edges(data=True))
    x.append(0)
    y.append(sum(d["length"] for u, v, d in largest_component.edges(data=True)) * 1. / tl)
    r = 0.0
    for i in range(1, n-1):
        g.remove_node(l.pop(0)[0])
        if recalculate:
            m = node_classifier(g)
            l = sorted(m.items(), key=operator.itemgetter(1),reverse=True)
            
        sbg = [g.subgraph(c).copy() for c in nx.connected_components(g)]
        largest_component = max(sbg, key=lambda c: sum(d["length"] for u, v, d in c.edges(data=True)))        
        x.append(i * 1. / n)
        r += sum(d["length"] for u, v, d in largest_component.edges(data=True)) * 1. / tl
        y.append(sum(d["length"] for u, v, d in largest_component.edges(data=True)) * 1. / tl)
    print(r,tl,n)
    return x, y, r / n

In [ ]:
def robustness_analysis_apl(g, node_classifier, recalculate=False):
    m = node_classifier(g)
    l = sorted(m.items(), key=operator.itemgetter(1), reverse=True)
    x = []
    y = []

    average_path_length = 0.0
    number_of_components = 0
    n = len(g.nodes())
    
    
    
    sbg = [g.subgraph(c).copy() for c in nx.connected_components(g)]
    for sg in sbg:
        average_path_length += nx.average_shortest_path_length(sg)
        number_of_components += 1

    average_path_length = average_path_length / number_of_components
    initial_apl = average_path_length

    x.append(0)
    y.append(average_path_length * 1. / initial_apl)
    r = 0.0
    for i in range(1, n-1):
        g.remove_node(l.pop(0)[0])
        if recalculate:
            m = node_classifier(g)
            l = sorted(m.items(), key=operator.itemgetter(1),
                       reverse=True)

        average_path_length = 0.0
        number_of_components = 0

        
        
        sbg = [g.subgraph(c).copy() for c in nx.connected_components(g)]
        for sg in sbg:
            if len(sg.nodes()) > 1:
                average_path_length += nx.average_shortest_path_length(sg)
            number_of_components += 1

        average_path_length = average_path_length / number_of_components

        x.append(i * 1. / n)
        r += average_path_length * 1. / initial_apl
        y.append(average_path_length * 1. / initial_apl)
    return x, y, r / n


In [ ]:

def random_ranking(g):
    nodes = g.nodes()
    values = g.nodes()
    values = list(values)
    random.shuffle(values)    
    
    return dict(zip(nodes, values))


In [ ]:
# network getting

# get 2021-04-01 bike networks(.graphml file and .png file) of Roma


ox.config(overpass_settings='[out:json][timeout:180][date:"2021-04-01T00:00:00Z"]')

place = 'Roma, Italy'

cf1 = '["highway"~"cycleway"]["bicycle"!~"no"]'
cf2 = '["cycleway"]["cycleway"!~"no"]'
cf3 = '["cycleway:right"]["cycleway:right"!~"no"]'
cf4 = '["cycleway:left"]["cycleway:left"!~"no"]'
cf5 = '["cycleway:both"]["cycleway:both"!~"no"]'
cf6 = '["bicycle"~"designated"]'
cf7 = '["oneway:bicycle"]'




G_21_b1=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf1)
G_21_b2=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf2)
G_21_b3=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf3)
G_21_b4=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf4)
G_21_b5=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf5)
G_21_b6=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf6)
G_21_b7=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf7)

G_21_b11 = nx.compose(G_21_b1,  G_21_b2)
G_21_b12 = nx.compose(G_21_b11, G_21_b3)
G_21_b13 = nx.compose(G_21_b12, G_21_b4)
G_21_b14 = nx.compose(G_21_b13, G_21_b5)
G_21_b15 = nx.compose(G_21_b14, G_21_b6)
G_21_16 = nx.compose(G_21_b15, G_21_b7)
G_21_16 = ox.simplification.simplify_graph(G_21_16)


G_21_21 = ox.simplification.consolidate_intersections(ox.project_graph(G_21_16,to_crs='EPSG:3812'), tolerance=15, rebuild_graph=True, dead_ends=True, reconnect_edges=True)
G_21_22 = ox.project_graph(G_21_21, to_crs='EPSG:4326')

fig, axb2 = ox.plot_graph(G_21_22, node_size=0,bgcolor='black',edge_color='white',edge_linewidth=0.5)


In [ ]:
# network getting

# get 2020-01-01 bike networks(.graphml file and .png file) of Roma


ox.config(overpass_settings='[out:json][timeout:180][date:"2020-01-01T00:00:00Z"]')

place = 'Roma, Italy'

cf1 = '["highway"~"cycleway"]["bicycle"!~"no"]'
cf2 = '["cycleway"]["cycleway"!~"no"]'
cf3 = '["cycleway:right"]["cycleway:right"!~"no"]'
cf4 = '["cycleway:left"]["cycleway:left"!~"no"]'
cf5 = '["cycleway:both"]["cycleway:both"!~"no"]'
cf6 = '["bicycle"~"designated"]'
cf7 = '["oneway:bicycle"]'



G_20_b1=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf1)
G_20_b2=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf2)
G_20_b3=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf3)


G_20_b6=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf6)
G_20_b7=ox.graph.graph_from_place(place, simplify=False, retain_all=True, 
                                  truncate_by_edge=True, which_result=None, buffer_dist=None, clean_periphery=True, 
                                  custom_filter = cf7)

G_20_b11 = nx.compose(G_20_b1,  G_20_b2)
G_20_b12 = nx.compose(G_20_b11, G_20_b3)
G_20_b15 = nx.compose(G_20_b12, G_20_b6)
G_20_16 = nx.compose(G_20_b15, G_20_b7)
G_20_16 = ox.simplification.simplify_graph(G_20_16)


G_20_21 = ox.simplification.consolidate_intersections(ox.project_graph(G_20_16,to_crs='EPSG:3812'), tolerance=15, rebuild_graph=True, dead_ends=True, reconnect_edges=True)
G_20_22 = ox.project_graph(G_20_21, to_crs='EPSG:4326')

fig, axb2 = ox.plot_graph(G_20_22, node_size=0,bgcolor='black',edge_color='white',edge_linewidth=0.5)

print('Roma 20_bike_net saved')




In [ ]:
apl=True

if apl==True:
    analysis_method = robustness_analysis_apl
    method='apl'
else:
    analysis_method = robustness_analysis
    method='com'
        
outfile='data_indicator/Roma/Robustness/part1/part1_b21_{}.png'.format(method)

recalculate=False

g = ox.utils_graph.get_digraph(G_21_22, weight='length')
g = g.to_undirected()
#g = ox.utils_graph.get_undirected(G_bspf)


x1, y1, vd = analysis_method(g.copy(), nx.degree_centrality, recalculate)
x2, y2, vb = analysis_method(g.copy(), nx.betweenness_centrality, recalculate)
x3, y3, vc = analysis_method(g.copy(), nx.closeness_centrality, recalculate)
x5, y5, vr = analysis_method(g.copy(), random_ranking)

if not apl:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Fractional size of largest component ($\sigma$)")
else:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Average path length ($\sigma$)")

pylab.plot(x1, y1, "b-", alpha=0.6, linewidth=2.0)
pylab.plot(x2, y2, "g-", alpha=0.6, linewidth=2.0)
pylab.plot(x3, y3, "r-", alpha=0.6, linewidth=2.0)
pylab.plot(x5, y5, "k-", alpha=0.6, linewidth=2.0)

# Generate csv file
import numpy as np

pylab.legend((r"Degree ($R = %4.3f$)" % vd,
              "Betweenness ($R = %4.3f$)" % vb,
              "Closeness ($R = %4.3f$)" % vc,
              "Random ($R = %4.3f$)" % vr),
            loc="upper right", shadow=False)

pylab.savefig(outfile, format="png")
pylab.close(1)

matrix = np.matrix([x1, y1, y2, y3, y5])
filename = outfile.rsplit(".", 1)[0] + ".csv"
header = " , degree, betweeness, closeness, random"
separator = ", "

np.savetxt(filename, matrix.transpose(), fmt="%2.5f", delimiter=separator,header=header, comments="")



In [ ]:
apl=True

if apl==True:
    analysis_method = robustness_analysis_apl
    method='apl'
else:
    analysis_method = robustness_analysis
    method='com'
        
outfile='data_indicator/Roma/Robustness/part1/part1_b20_{}.png'.format(method)

recalculate=False

g = ox.utils_graph.get_digraph(G_20_22, weight='length')
g = g.to_undirected()
#g = ox.utils_graph.get_undirected(G_bspf)


x1, y1, vd = analysis_method(g.copy(), nx.degree_centrality, recalculate)
x2, y2, vb = analysis_method(g.copy(), nx.betweenness_centrality, recalculate)
x3, y3, vc = analysis_method(g.copy(), nx.closeness_centrality, recalculate)
x5, y5, vr = analysis_method(g.copy(), random_ranking)

if not apl:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Fractional size of largest component ($\sigma$)")
else:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Average path length ($\sigma$)")

pylab.plot(x1, y1, "b-", alpha=0.6, linewidth=2.0)
pylab.plot(x2, y2, "g-", alpha=0.6, linewidth=2.0)
pylab.plot(x3, y3, "r-", alpha=0.6, linewidth=2.0)
pylab.plot(x5, y5, "k-", alpha=0.6, linewidth=2.0)

# Generate csv file
import numpy as np

pylab.legend((r"Degree ($R = %4.3f$)" % vd,
              "Betweenness ($R = %4.3f$)" % vb,
              "Closeness ($R = %4.3f$)" % vc,
              "Random ($R = %4.3f$)" % vr),
            loc="upper right", shadow=False)

pylab.savefig(outfile, format="png")
pylab.close(1)

matrix = np.matrix([x1, y1, y2, y3, y5])
filename = outfile.rsplit(".", 1)[0] + ".csv"
header = " , degree, betweeness, closeness, random"
separator = ", "

np.savetxt(filename, matrix.transpose(), fmt="%2.5f", delimiter=separator,header=header, comments="")



In [ ]:
apl=False

if apl==True:
    analysis_method = robustness_analysis_apl
    method='apl'
else:
    analysis_method = robustness_analysis
    method='com'
        
outfile='data_indicator/Roma/Robustness/part1/part1_b21_{}.png'.format(method)

recalculate=False

g = ox.utils_graph.get_digraph(G_21_22, weight='length')
g = g.to_undirected()
#g = ox.utils_graph.get_undirected(G_bspf)


x1, y1, vd = analysis_method(g.copy(), nx.degree_centrality, recalculate)
x2, y2, vb = analysis_method(g.copy(), nx.betweenness_centrality, recalculate)
x3, y3, vc = analysis_method(g.copy(), nx.closeness_centrality, recalculate)
x5, y5, vr = analysis_method(g.copy(), random_ranking)

if not apl:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Fractional size of largest component ($\sigma$)")
else:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Average path length ($\sigma$)")

pylab.plot(x1, y1, "b-", alpha=0.6, linewidth=2.0)
pylab.plot(x2, y2, "g-", alpha=0.6, linewidth=2.0)
pylab.plot(x3, y3, "r-", alpha=0.6, linewidth=2.0)
pylab.plot(x5, y5, "k-", alpha=0.6, linewidth=2.0)

# Generate csv file
import numpy as np

pylab.legend((r"Degree ($R = %4.3f$)" % vd,
              "Betweenness ($R = %4.3f$)" % vb,
              "Closeness ($R = %4.3f$)" % vc,
              "Random ($R = %4.3f$)" % vr),
            loc="upper right", shadow=False)

pylab.savefig(outfile, format="png")
pylab.close(1)

matrix = np.matrix([x1, y1, y2, y3, y5])
filename = outfile.rsplit(".", 1)[0] + ".csv"
header = " , degree, betweeness, closeness, random"
separator = ", "

np.savetxt(filename, matrix.transpose(), fmt="%2.5f", delimiter=separator,header=header, comments="")



In [ ]:
apl=False

if apl==True:
    analysis_method = robustness_analysis_apl
    method='apl'
else:
    analysis_method = robustness_analysis
    method='com'
        
outfile='data_indicator/Roma/Robustness/part1/part1_b20_{}.png'.format(method)

recalculate=False

g = ox.utils_graph.get_digraph(G_20_22, weight='length')
g = g.to_undirected()
#g = ox.utils_graph.get_undirected(G_bspf)


x1, y1, vd = analysis_method(g.copy(), nx.degree_centrality, recalculate)
x2, y2, vb = analysis_method(g.copy(), nx.betweenness_centrality, recalculate)
x3, y3, vc = analysis_method(g.copy(), nx.closeness_centrality, recalculate)
x5, y5, vr = analysis_method(g.copy(), random_ranking)

if not apl:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Fractional size of largest component ($\sigma$)")
else:
    pylab.figure(1, dpi=1080)
    pylab.xlabel(r"Fraction of vertices removed ($\rho$)")
    pylab.ylabel(r"Average path length ($\sigma$)")

pylab.plot(x1, y1, "b-", alpha=0.6, linewidth=2.0)
pylab.plot(x2, y2, "g-", alpha=0.6, linewidth=2.0)
pylab.plot(x3, y3, "r-", alpha=0.6, linewidth=2.0)
pylab.plot(x5, y5, "k-", alpha=0.6, linewidth=2.0)

# Generate csv file
import numpy as np

pylab.legend((r"Degree ($R = %4.3f$)" % vd,
              "Betweenness ($R = %4.3f$)" % vb,
              "Closeness ($R = %4.3f$)" % vc,
              "Random ($R = %4.3f$)" % vr),
            loc="upper right", shadow=False)

pylab.savefig(outfile, format="png")
pylab.close(1)

matrix = np.matrix([x1, y1, y2, y3, y5])
filename = outfile.rsplit(".", 1)[0] + ".csv"
header = " , degree, betweeness, closeness, random"
separator = ", "

np.savetxt(filename, matrix.transpose(), fmt="%2.5f", delimiter=separator,header=header, comments="")

